In [1]:
!pip install fvcore
!pip install open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=3637769b8001eef423a624ddd5095aadf62de65d5739eea4c1c69018003487f2
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=c5b0a19572a575da0c09c7fd5d3db3749529dd0c1a06d748dcad2c016077984b
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

test_df = pd.read_csv('/kaggle/input/grand-xray-slam-division-b/sample_submission_2.csv')

In [3]:
test_images_folder='/kaggle/input/600p-div-b-data/test2_resized'

chexfound_folder_path='/kaggle/input/models-all-data-chexfound-recs-xray-div-b'

evax448_folder_path_ep5='/kaggle/input/evax-recs-448-xray-div-b'
evax448_folder_path_ep6='/kaggle/input/evax-recs-448-xray-div-b'

evax224_folder_path='/kaggle/input/evax-recs-xray-div-b'

In [4]:
labels_cols=[
            "Atelectasis",
            "Cardiomegaly",
            "Consolidation",
            "Edema",
            "Enlarged Cardiomediastinum",
            "Fracture",
            "Lung Lesion",
            "Lung Opacity",
            "No Finding",
            "Pleural Effusion",
            "Pleural Other",
            "Pneumonia",
            "Pneumothorax",
            "Support Devices",
        ]

In [5]:
import sys

repo_path = f"{chexfound_folder_path}/CheXFound"
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

In [6]:
import os

#os.chdir('/kaggle/working/CheXFound')  # Change this path to the repository path
import json
import torch
import argparse
import pandas as pd
from PIL import Image
from chexfound.eval.setup import setup_and_build_model
from chexfound.data.transforms import make_classification_eval_transform
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from chexfound.eval.utils import extract_hyperparameters_from_model
from chexfound.eval.classification.utils import setup_glori
from fvcore.common.checkpoint import Checkpointer
from matplotlib.colors import ListedColormap

import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchvision.io as io
from sklearn.model_selection import StratifiedShuffleSplit, KFold
from torch.amp import autocast, GradScaler
from torchvision.transforms import autoaugment, transforms
from transformers import get_cosine_schedule_with_warmup
import copy
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score
from timm.models.eva import Eva
from timm.layers import resample_abs_pos_embed, resample_patch_embed

/kaggle/input/models-all-data-chexfound-recs-xray-div-b/CheXFound/chexfound/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/kaggle/input/models-all-data-chexfound-recs-xray-div-b/CheXFound/chexfound/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/kaggle/input/models-all-data-chexfound-recs-xray-div-b/CheXFound/chexfound/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
2025-10-11 10:28:42.894692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760178523.120549      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# chexfound

In [8]:
base_dir = f'{chexfound_folder_path}/CheXFound_Model/'  # Change to the directory storing checkpoints and configuration files

config_file = base_dir + 'config.yaml'
pretrained_weights = base_dir + 'teacher_checkpoint.pth'
classifier_fpath = base_dir + 'glori.pth'
classifier_json = base_dir + 'results_eval_linear.json'
output_dir = base_dir + 'example'
os.makedirs(output_dir, exist_ok=True)

In [9]:
parser = argparse.ArgumentParser()

parser.set_defaults(
    config_file=config_file,  # path to architecture configuration files
    pretrained_weights=None,
    output_dir=output_dir,
    opts=[],
    image_size=512,
    patch_size=16,
    n_register_tokens=4,
    n_last_blocks=4,
    return_class_token=True,
    num_classes=40,
    num_heads=8,
)
args, unknown = parser.parse_known_args()

In [10]:
!git config --global --add safe.directory /kaggle/input/models-all-data-chexfound-recs-xray-div-b/CheXFound

In [11]:
args.output_dir = ""

#os.makedirs(args.output_dir, exist_ok=True)
base_model, autocast_dtype = setup_and_build_model(args)

[rank0]:[W1011 10:28:59.430209621 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


I20251011 10:29:00 19 chexfound config.py:60] git:
  sha: a4bac4956f3eeba4a8b2c15410c600536c7b0bce, status: has uncommitted changes, branch: main

I20251011 10:29:00 19 chexfound config.py:61] config_file: /kaggle/input/models-all-data-chexfound-recs-xray-div-b/CheXFound_Model/config.yaml
image_size: 512
n_last_blocks: 4
n_register_tokens: 4
num_classes: 40
num_heads: 8
opts: ['train.output_dir=/kaggle/working']
output_dir: /kaggle/working
patch_size: 16
pretrained_weights: None
return_class_token: True
I20251011 10:29:00 19 chexfound config.py:26] sqrt scaling learning rate; base: 0.0002, new: 2.3385358667337133e-05
I20251011 10:29:00 19 chexfound config.py:33] MODEL:
  WEIGHTS: ''
compute_precision:
  grad_scaler: true
  teacher:
    backbone:
      sharding_strategy: SHARD_GRAD_OP
      mixed_precision:
        param_dtype: fp16
        reduce_dtype: fp16
        buffer_dtype: fp32
    dino_head:
      sharding_strategy: SHARD_GRAD_OP
      mixed_precision:
        param_dtype: fp16

In [12]:
# set up glori head
log_json = classifier_json
with open(log_json, 'r') as f:
    content = f.read().split('\n')[-3]
    data = json.loads(content)
best_classifier_str = data['best_classifier']['name']
hyperparameters = extract_hyperparameters_from_model(best_classifier_str)
learning_rate, avgpool, block = hyperparameters["lr"], hyperparameters["avgpool"], hyperparameters["blocks"]

sample_input = torch.randn(1, 3, 512, 512).cuda()
with torch.no_grad():
    sample_output = base_model.get_intermediate_layers(sample_input, n=args.n_last_blocks, return_class_token=True)

glori, _ = setup_glori(
    sample_output=sample_output,
    n_last_blocks_list=block,
    learning_rates=learning_rate,
    avgpools=avgpool,
    num_classes=args.num_classes,
    multiview=False,
    decoder_dim=768,
    cat_cls=False,
)

In [13]:
# ---------- Fix decoder internals (resize biases / num_classes) ----------
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TARGET_CLASSES = 14
ORIG_CLASSES = 40

# get classifier same way as before
glori_wrapped = glori
glori_module = getattr(glori_wrapped, "module", glori_wrapped)
classifier_key = next(iter(glori_module.classifiers_dict.keys()))
classifier = glori_module.classifiers_dict[classifier_key]
decoder = getattr(classifier, "decoder", None)
if decoder is None:
    raise RuntimeError("Decoder not found in classifier; cannot resize num_classes. Print classifier to inspect.")

print("Decoder found:", type(decoder))
print("Decoder.num_classes (before):", getattr(decoder, "num_classes", None))

# 1) set decoder.num_classes
decoder.num_classes = TARGET_CLASSES
print("Set decoder.num_classes ->", decoder.num_classes)

# 2) replace duplicate_pooling_bias if exists (common in this implementation)
if hasattr(decoder, "duplicate_pooling_bias"):
    old = decoder.duplicate_pooling_bias
    print("Old duplicate_pooling_bias shape:", tuple(old.shape) if isinstance(old, torch.Tensor) else type(old))
    # create new bias param
    new_bias = nn.Parameter(torch.zeros(TARGET_CLASSES, dtype=torch.float32, device=device))
    # init small (optional)
    nn.init.constant_(new_bias, 0.0)
    # assign (if module originally on CPU, move)
    decoder.duplicate_pooling_bias = new_bias
    print("Replaced duplicate_pooling_bias with shape:", decoder.duplicate_pooling_bias.shape)
else:
    print("No attribute duplicate_pooling_bias found on decoder — fine.")

# 3) scan decoder for any 1D parameters/buffers whose length equals ORIG_CLASSES and replace them (safe heuristic)
replaced = []
for name, param in list(decoder.named_parameters()):
    if param.dim() == 1 and param.shape[0] == ORIG_CLASSES:
        print(f"Resizing param: {name} shape {param.shape} -> {TARGET_CLASSES}")
        parent = decoder
        parts = name.split('.')
        for p in parts[:-1]:
            parent = getattr(parent, p)
        attr = parts[-1]
        new_p = nn.Parameter(torch.zeros(TARGET_CLASSES, dtype=param.dtype, device=device))
        setattr(parent, attr, new_p)
        replaced.append(name)

# also check buffers (non-parameter tensors)
for name, buf in list(decoder.named_buffers()):
    if buf is None:
        continue
    if buf.ndim == 1 and buf.shape[0] == ORIG_CLASSES:
        print(f"Resizing buffer: {name} shape {buf.shape} -> {TARGET_CLASSES}")
        parent = decoder
        parts = name.split('.')
        for p in parts[:-1]:
            parent = getattr(parent, p)
        attr = parts[-1]
        # assign as buffer
        decoder.register_buffer(attr, torch.zeros(TARGET_CLASSES, dtype=buf.dtype, device=device))
        replaced.append("buffer:"+name)

print("Replaced parameter/buffer names:", replaced)

# 4) move whole glori to device again and do a quick forward test
glori_wrapped.to(device)
print("Moved glori to device.")

Decoder found: <class 'chexfound.eval.classification.glori.TransformerDecoder'>
Decoder.num_classes (before): 40
Set decoder.num_classes -> 14
Old duplicate_pooling_bias shape: (40,)
Replaced duplicate_pooling_bias with shape: torch.Size([14])
Replaced parameter/buffer names: []
Moved glori to device.


In [14]:
# 6) Wrap backbone + new glori into a training model
class CheXFoundWithGLoRIHead(nn.Module):
    def __init__(self, backbone, glori_module, args):
        super().__init__()
        self.backbone = backbone
        self.glori = glori_module
        self.args = args
    def forward(self, x):
        # get intermediate features and pass to glori
        feats = self.backbone.get_intermediate_layers(x, n=self.args.n_last_blocks, return_class_token=self.args.return_class_token)
        logits_dict = self.glori([feats], return_attention=False)
        key = list(logits_dict.keys())[0]
        return logits_dict[key]

In [15]:
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2


cv2.setNumThreads(0)

class TestChestXrayDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.label_cols = [
            "Atelectasis","Cardiomegaly","Consolidation","Edema",
            "Enlarged Cardiomediastinum","Fracture","Lung Lesion",
            "Lung Opacity","No Finding","Pleural Effusion","Pleural Other",
            "Pneumonia","Pneumothorax","Support Devices",
        ]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row["Image_name"]
        img_path = os.path.join(self.img_dir, row["Image_name"])

        # Fast read with OpenCV. It may return None if file missing -> handle.
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")

        # If grayscale (H, W), convert to 3-channel
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:
            # if RGBA, convert to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        # OpenCV loads BGR -> convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=img)
            img = augmented["image"]

        return img, img_id

# choose img_size once
img_size = 512
size_tuple = (img_size, img_size)   # IMPORTANT: albumentations expects a tuple

val_tfms = A.Compose([
    A.Resize(height=img_size, width=img_size),   # pass height & width
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [16]:
class ModelEMA:
    def __init__(self, model, decay=0.9997, device=None):
        # copy *unwrapped* module (so EMA keys match saved/loaded model keys)
        self.decay = decay
        self.device = device
        self.ema = copy.deepcopy(get_model(model)).eval()
        if device is not None:
            self.ema.to(device)
        for p in self.ema.parameters():
            p.requires_grad_(False)

    def update(self, model):
        src_state = get_model(model).state_dict()
        with torch.no_grad():
            for k, v in self.ema.state_dict().items():
                model_v = src_state[k].detach().to(v.device)
                v.copy_(v * self.decay + (1.0 - self.decay) * model_v)

    def state_dict(self):
        return self.ema.state_dict()


def get_model(module):
    """Return the underlying model (unwrap DataParallel / DDP)."""
    return module.module if hasattr(module, "module") else module

In [17]:
scaler = GradScaler(device="cuda")

def predict_chexfound(dataset,base_model,model_path='', batch_size=128, device="cuda"):
    fold_probs = []
    all_ids = None
    
    
    
    with torch.no_grad():
        
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        
        # -----------------------------
        # Model: ConvNeXt
        # -----------------------------
        # -------------------------
        
        # SAFE unwrap & prepare model for multi-GPU
        # -------------------------
        print("Preparing model on device:", DEVICE, "GPUs:", torch.cuda.device_count())
        
        # 1) If glori is a DDP wrapper, unwrap to get the real module
        glori_wrapped = glori  # original object you loaded
        if hasattr(glori_wrapped, "module"):
            print("Unwrapping glori from DDP wrapper -> using glori.module")
            glori_clean = glori_wrapped.module
        else:
            glori_clean = glori_wrapped
        
        # move glori_clean to CPU to avoid device mismatch while assembling
        glori_clean = glori_clean.cpu()
        
        # 2) Rebuild train_model with the unwrapped glori (fresh object)
        model = CheXFoundWithGLoRIHead(backbone=base_model, glori_module=glori_clean, args=args)
        
        model = model 
        checkpoint = torch.load(
            model_path,
            map_location="cpu",
            weights_only=False  
        )
        model.load_state_dict(checkpoint["ema_state_dict"])

        
        
        # 5) Move model to device
        model = model.to(DEVICE)
        # 4) If you have multiple GPUs, wrap with nn.DataParallel (NOT DDP)
        if torch.cuda.device_count() > 1:
            print("Wrapping train_model with nn.DataParallel for multi-GPU")
            model = nn.DataParallel(model)
        
        
        
        # IMPORTANT: make sure the name used in the rest of your loop refers to the wrapped model
        

        model.eval()
        

        fold_out = []
        ids = []

        with torch.no_grad():
            for images, img_id in tqdm(loader, desc=f"Inference Fold "):
                images = images.to(device)
                with autocast(device_type="cuda"):
                    logits = model(images)
                probs = torch.sigmoid(logits)
                fold_out.append(probs.cpu())
                if all_ids is None:  # only collect IDs once
                    ids.extend(img_id)

        fold_out = torch.cat(fold_out, dim=0)  # [N, C]
        fold_probs.append(fold_out)

        if all_ids is None:
            all_ids = ids

        final_probs = torch.stack(fold_probs, dim=0).mean(0)  # [N, C]
        return all_ids, final_probs

In [ ]:
test_dataset = TestChestXrayDataset(
    df=test_df,
    img_dir=test_images_folder,
    transform=val_tfms
)
all_ids,final_probs = predict_chexfound(test_dataset,base_model,model_path=f'{chexfound_folder_path}/CheXFound/ema_model_f5.pth', batch_size=64, device=DEVICE)



all_preds = np.vstack(final_probs)

# -----------------------------
# Build Submission
# -----------------------------
sample_df = test_df

submission_chexfound_ep5 = pd.DataFrame(all_preds, columns=labels_cols)
submission_chexfound_ep5.insert(0, "Image_name", all_ids)

# 🔒 Align with sample submission
submission_chexfound_ep5 = submission_chexfound_ep5[sample_df.columns]

# assert submission_chexfound_ep5.shape == sample_df.shape, "Shape mismatch with sample submission!"
# assert all(submission_chexfound_ep5["Image_name"] == sample_df["Image_name"]), "Image names mismatch!"

# submission_chexfound_ep5.to_csv("submission_chexfound_ep5.csv", index=False)
# print("✅ Submission file saved with correct format")

Preparing model on device: cuda GPUs: 2
Unwrapping glori from DDP wrapper -> using glori.module
Wrapping train_model with nn.DataParallel for multi-GPU


Inference Fold : 100%|██████████| 749/749 [1:20:20<00:00,  6.44s/it]


✅ Submission file saved with correct format


In [ ]:
test_dataset = TestChestXrayDataset(
    df=test_df,
    img_dir=test_images_folder,
    transform=val_tfms
)
all_ids,final_probs = predict_chexfound(test_dataset,base_model,model_path=f'{chexfound_folder_path}/CheXFound/ema_model_f6.pth', batch_size=64, device=DEVICE)


all_preds = np.vstack(final_probs)

# -----------------------------
# Build Submission
# -----------------------------
sample_df = test_df

submission_chexfound_ep6 = pd.DataFrame(all_preds, columns=labels_cols)
submission_chexfound_ep6.insert(0, "Image_name", all_ids)

# 🔒 Align with sample submission
submission_chexfound_ep6 = submission_chexfound_ep6[sample_df.columns]

# assert submission_chexfound_ep6.shape == sample_df.shape, "Shape mismatch with sample submission!"
# assert all(submission_chexfound_ep6["Image_name"] == sample_df["Image_name"]), "Image names mismatch!"

# submission_chexfound_ep6.to_csv("submission_chexfound_ep6.csv", index=False)
# print("✅ Submission file saved with correct format")

Preparing model on device: cuda GPUs: 2
Unwrapping glori from DDP wrapper -> using glori.module
Wrapping train_model with nn.DataParallel for multi-GPU


Inference Fold : 100%|██████████| 749/749 [1:21:14<00:00,  6.51s/it]


✅ Submission file saved with correct format


In [22]:
torch.cuda.empty_cache()

# evax 448

In [23]:
def checkpoint_filter_fn(
        state_dict,
        model,
        interpolation='bicubic',
        antialias=True,
):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    state_dict = state_dict.get('model_ema', state_dict)
    state_dict = state_dict.get('model', state_dict)
    state_dict = state_dict.get('module', state_dict)
    state_dict = state_dict.get('state_dict', state_dict)
    # prefix for loading OpenCLIP compatible weights
    if 'visual.trunk.pos_embed' in state_dict:
        prefix = 'visual.trunk.'
    elif 'visual.pos_embed' in state_dict:
        prefix = 'visual.'
    else:
        prefix = ''
    mim_weights = prefix + 'mask_token' in state_dict
    no_qkv = prefix + 'blocks.0.attn.q_proj.weight' in state_dict

    len_prefix = len(prefix)
    for k, v in state_dict.items():
        if prefix:
            if k.startswith(prefix):
                k = k[len_prefix:]
            else:
                continue

        if 'rope' in k:
            # fixed embedding no need to load buffer from checkpoint
            continue

        if 'patch_embed.proj.weight' in k:
            _, _, H, W = model.patch_embed.proj.weight.shape
            if v.shape[-1] != W or v.shape[-2] != H:
                v = resample_patch_embed(
                    v,
                    (H, W),
                    interpolation=interpolation,
                    antialias=antialias,
                    verbose=True,
                )
        elif k == 'pos_embed' and v.shape[1] != model.pos_embed.shape[1]:
            # To resize pos embedding when using model at different size from pretrained weights
            num_prefix_tokens = 0 if getattr(model, 'no_embed_class', False) else getattr(model, 'num_prefix_tokens', 1)
            v = resample_abs_pos_embed(
                v,
                new_size=model.patch_embed.grid_size,
                num_prefix_tokens=num_prefix_tokens,
                interpolation=interpolation,
                antialias=antialias,
                verbose=True,
            )

        k = k.replace('mlp.ffn_ln', 'mlp.norm')
        k = k.replace('attn.inner_attn_ln', 'attn.norm')
        k = k.replace('mlp.w12', 'mlp.fc1')
        k = k.replace('mlp.w1', 'mlp.fc1_g')
        k = k.replace('mlp.w2', 'mlp.fc1_x')
        k = k.replace('mlp.w3', 'mlp.fc2')
        if no_qkv:
            k = k.replace('q_bias', 'q_proj.bias')
            k = k.replace('v_bias', 'v_proj.bias')

        if mim_weights and k in ('mask_token', 'lm_head.weight', 'lm_head.bias', 'norm.weight', 'norm.bias'):
            if k == 'norm.weight' or k == 'norm.bias':
                # try moving norm -> fc norm on fine-tune, probably a better starting point than new init
                k = k.replace('norm', 'fc_norm')
            else:
                # skip pretrain mask token & head weights
                continue

        out_dict[k] = v

    return out_dict

class EVA_X(Eva):
    def __init__(self, **kwargs):
        super(EVA_X, self).__init__(**kwargs)

    def forward_features(self, x):
        x = self.patch_embed(x)
        x, rot_pos_embed = self._pos_embed(x)
        for blk in self.blocks:
            x = blk(x, rope=rot_pos_embed)
        x = self.norm(x)
        return x

    def forward_head(self, x, pre_logits: bool = False):
        if self.global_pool:
            x = x[:, self.num_prefix_tokens:].mean(dim=1) if self.global_pool == 'avg' else x[:, 0]
        x = self.fc_norm(x)
        x = self.head_drop(x)
        return x if pre_logits else self.head(x)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.forward_head(x)
        return x


def eva_x_base_patch16(pretrained=False):
    model = EVA_X(
        img_size=img_size,
        patch_size=16,
        embed_dim=768,
        depth=12,
        num_heads=12,
        qkv_fused=False,
        mlp_ratio=4 * 2 / 3,
        swiglu_mlp=True,
        scale_mlp=True,
        use_rot_pos_emb=True,
        ref_feat_shape=(14, 14),  # 224/16
    )
    eva_ckpt = checkpoint_filter_fn(torch.load(pretrained, map_location='cpu', weights_only=False), 
                        model)
    msg = model.load_state_dict(eva_ckpt, strict=False)
    print(msg)
    return model

In [24]:
#evax base
class EVAX_Model(nn.Module):
    def __init__(self, unfreeze_last_n_blocks=4 ):
        super().__init__()
        # Load Phikon backbone
        eva_x_b_pt = '/kaggle/input/evax-recs-448-xray-div-b/eva_x_base_patch16_merged520k_mim.pt'  
        self.backbone = eva_x_base_patch16(pretrained=eva_x_b_pt)
        
        self.backbone.head=nn.Linear(in_features=768, out_features=14, bias=True)

        # # Also unfreeze head and normalisation layers around it
        for p in self.backbone.head.parameters():
            p.requires_grad = True

    
    
    def forward(self, x):
        backbone_output = self.backbone(x)
    
        #return self.head(backbone_output)
        return backbone_output

In [25]:
img_size = 448

normalize = A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])

def scale_brightness(factor):
    def apply_fn(x, **kwargs):
        return (x * factor).clip(0, 255).astype(x.dtype)
    return apply_fn

tta_transforms = [

    # 1. Original
    A.Compose([
        A.Resize(img_size, img_size),
        normalize,
        ToTensorV2(),
    ]),

    # 2. Horizontal Flip
    A.Compose([
        A.HorizontalFlip(p=1.0),
        A.Resize(img_size, img_size),
        normalize,
        ToTensorV2(),
    ]),

    # 3–5. Multi-scale center crops (zoom in/out)
    *[
        A.Compose([
            A.Resize(int(img_size * scale), int(img_size * scale)),
            A.CenterCrop(img_size, img_size),
            normalize,
            ToTensorV2(),
        ])
        #for scale in [0.9, 1.0, 1.1]
        for scale in [1.0, 1.1]
    ],

    # 6–7. Brightness scaling
    A.Compose([
        A.Resize(img_size, img_size),
        A.Lambda(image=scale_brightness(0.95)),
        normalize,
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(img_size, img_size),
        A.Lambda(image=scale_brightness(1.05)),
        normalize,
        ToTensorV2(),
    ]),
]

In [26]:
scaler = GradScaler(device="cuda")

def predict_tta(dataset, tta_transforms, model_path='', batch_size=128, device="cuda"):
    fold_probs = []
    all_ids = None
    
    #k=1

    # -----------------------------
    # Model: ConvNeXt
    # -----------------------------
    model = EVAX_Model()
    checkpoint = torch.load(
        model_path,
        map_location="cpu",
        weights_only=False  
    )
    model.load_state_dict(checkpoint["ema_state_dict"])
    
    if torch.cuda.device_count() > 1:
        print(f"⚡ Using {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)  # wrap for multi-GPU
    
    model = model.to(DEVICE)
    

    model.eval()
    

    ids = []
    
    for tfm in tta_transforms:
        dataset.transform = tfm 
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        
        fold_out = []
        with torch.no_grad():
            for images, img_id in tqdm(loader, desc=f"Inference"):
                
                images = images.to(device)
                with autocast(device_type="cuda"):
                    logits = model(images)
                probs = torch.sigmoid(logits)
                fold_out.append(probs.cpu())
                if all_ids is None:  # only collect IDs once
                    ids.extend(img_id)

        fold_out = torch.cat(fold_out, dim=0)  # [N, C]
        fold_probs.append(fold_out)

        if all_ids is None:
            all_ids = ids

    final_probs = torch.stack(fold_probs, dim=0).mean(0)  # [N, C]
    return all_ids, final_probs

In [ ]:
test_dataset = TestChestXrayDataset(
    df=test_df,
    img_dir=test_images_folder,
    transform=None
)
all_ids,final_probs = predict_tta(test_dataset, tta_transforms, model_path=f'{evax448_folder_path_ep5}/ema_model_f5.pth', batch_size=128, device=DEVICE)

all_preds = np.vstack(final_probs)
# -----------------------------
# Build Submission
# -----------------------------
sample_df = test_df

submission_evax448_ep5 = pd.DataFrame(all_preds, columns=labels_cols)
submission_evax448_ep5.insert(0, "Image_name", all_ids)

# 🔒 Align with sample submission
submission_evax448_ep5 = submission_evax448_ep5[sample_df.columns]

# assert submission_evax448_ep5.shape == sample_df.shape, "Shape mismatch with sample submission!"
#assert all(submission_evax448_ep5["Image_name"] == sample_df["Image_name"]), "Image names mismatch!"

# submission_evax448_ep5.to_csv("submission_evax448_ep5.csv", index=False)
# print("✅ Submission file saved with correct format")

I20251011 13:11:11 19 timm.layers.pos_embed pos_embed.py:57] Resized position embedding: (14, 14) to (28, 28).
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
⚡ Using 2 GPUs


Inference: 100%|██████████| 375/375 [20:31<00:00,  3.28s/it]


✅ Submission file saved with correct format


In [ ]:
test_dataset = TestChestXrayDataset(
    df=test_df,
    img_dir=test_images_folder,
    transform=None
)
all_ids,final_probs = predict_tta(test_dataset, tta_transforms, model_path=f'{evax448_folder_path_ep6}/ema_model_f6.pth', batch_size=128, device=DEVICE)

all_preds = np.vstack(final_probs)
# -----------------------------
# Build Submission
# -----------------------------
sample_df = test_df

submission_evax448_ep6 = pd.DataFrame(all_preds, columns=labels_cols)
submission_evax448_ep6.insert(0, "Image_name", all_ids)

# 🔒 Align with sample submission
submission_evax448_ep6 = submission_evax448_ep6[sample_df.columns]

# assert submission_evax448_ep6.shape == sample_df.shape, "Shape mismatch with sample submission!"
# assert all(submission_evax448_ep6["Image_name"] == sample_df["Image_name"]), "Image names mismatch!"

# submission_evax448_ep6.to_csv("submission_evax448_ep6.csv", index=False)
# print("✅ Submission file saved with correct format")

I20251011 15:13:57 19 timm.layers.pos_embed pos_embed.py:57] Resized position embedding: (14, 14) to (28, 28).
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
⚡ Using 2 GPUs


Inference: 100%|██████████| 375/375 [20:35<00:00,  3.29s/it]


✅ Submission file saved with correct format


In [31]:
torch.cuda.empty_cache()

In [35]:
submission_evax224_kfold_tta

,Image_name,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000002_002_001.jpg,0.629883,0.583008,0.452637,0.448975,0.774414,0.667969,0.096558,0.818359,0.104797,0.366943,0.296631,0.304199,0.028595,0.138428
1,00000002_001_001.jpg,0.779297,0.589844,0.423340,0.371826,0.821777,0.621094,0.022598,0.858887,0.081787,0.554199,0.281250,0.205200,0.030396,0.268311
2,00000002_001_002.jpg,0.791016,0.725098,0.713867,0.592285,0.874023,0.671875,0.181396,0.933594,0.025803,0.628418,0.321533,0.468750,0.046814,0.319336
3,00000008_001_001.jpg,0.875000,0.768555,0.685547,0.527832,0.892578,0.476562,0.088257,0.926758,0.014206,0.787109,0.182251,0.290039,0.228271,0.758301
4,00000008_002_001.jpg,0.977539,0.945801,0.934570,0.748535,0.979980,0.245850,0.066406,0.991699,0.001653,0.945312,0.073547,0.296143,0.114807,0.933105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47922,20009239_009_000.jpg,0.134644,0.006367,0.029861,0.002016,0.001216,0.002432,0.274658,0.170776,0.631348,0.059235,0.184692,0.012726,0.047791,0.000591
47923,20009239_010_000.jpg,0.130371,0.003519,0.049347,0.007172,0.001210,0.001625,0.137695,0.138672,0.646484,0.232178,0.236938,0.010223,0.082031,0.000852
47924,20009239_011_000.jpg,0.109619,0.003643,0.042755,0.004852,0.001096,0.001647,0.127197,0.124329,0.685547,0.224976,0.221069,0.008209,0.058594,0.000761
47925,20009239_012_000.jpg,0.218262,0.008545,0.035095,0.004673,0.000968,0.001959,0.201050,0.203003,0.594238,0.091919,0.233398,0.018280,0.064453,0.000787


# ensemble

In [ ]:
# submission_chexfound_ep5 
# submission_chexfound_ep6 
# submission_evax448_ep5 
# submission_evax448_ep6 




# Average predictions (excluding Image_name)
avg_sub = submission_chexfound_ep5.copy()

#just chexfound and evax 448
avg_sub.iloc[:, 1:] = (submission_chexfound_ep5.iloc[:, 1:]+submission_chexfound_ep6.iloc[:, 1:] + submission_evax448_ep5.iloc[:, 1:] + submission_evax448_ep6.iloc[:, 1:]) / 4

# Save new submission
avg_sub.to_csv('submission.csv', index=False)

print("✅ Averaged submission saved as submission_avg.csv")
avg_sub.head()

✅ Averaged submission saved as submission_avg.csv


,Image_name,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000002_002_001.jpg,0.564387,0.596802,0.329891,0.434611,0.790131,0.489174,0.060137,0.758917,0.106129,0.336172,0.189307,0.186575,0.006555,0.093347
1,00000002_001_001.jpg,0.807116,0.625642,0.486450,0.358501,0.804823,0.656297,0.020160,0.859516,0.046346,0.642727,0.272219,0.232052,0.028079,0.270676
2,00000002_001_002.jpg,0.839652,0.737760,0.654369,0.507068,0.896529,0.694261,0.083625,0.939934,0.017534,0.615645,0.337896,0.410849,0.049263,0.238332
3,00000008_001_001.jpg,0.912232,0.817734,0.699534,0.566302,0.931919,0.553309,0.086039,0.961791,0.010881,0.816973,0.174218,0.347548,0.070342,0.670559
4,00000008_002_001.jpg,0.988560,0.975628,0.961297,0.862386,0.993055,0.136023,0.066298,0.998082,0.000524,0.965818,0.030735,0.308420,0.062013,0.955870
